# POC PUCOMEX - Atendimento Serpro

## Instalação de pacotes necessários

In [1]:
# Importações básicas - Descomentar as linhas abaixo:
#!pip install pandas
#!pip install tensorflow
#!pip install keras
#!pip install scikit-learn
#!pip install nltk
#!pip install ezodf
#!pip install lxml
#import nltk
#nltk.download('all')
#!pip install spacy
#!python -m spacy download pt_core_news_sm

## Importações e criação de funções

In [49]:
import ezodf
import pandas as pd
import sys
import unicodedata
import re
import time
import spacy
from unicodedata import normalize
from nltk import tokenize # Tokenização
from nltk.corpus import stopwords # Stopwords
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import RSLPStemmer
# Função para mostrar as planilhas do documento
def show_ods_sheets(filename):
    doc = ezodf.opendoc(filename)
    print("Spreadsheet contains %d sheet(s)." % len(doc.sheets))
    for sheet in doc.sheets:
        print("-"*40)
        print("   Sheet name : '%s'" % sheet.name)
        print("Size of Sheet : (rows=%d, cols=%d)" % (sheet.nrows(), sheet.ncols()) )
# Função para converter ODS em DataFrame do Pandas
def read_sheet_ods_as_panda_dataframe(filename, sheet_no=0, header=0):
    tab = ezodf.opendoc(filename=filename).sheets[sheet_no]
    return pd.DataFrame({col[header].value:[x.value for x in col[header+1:]]
                         for col in tab.columns()})

## Abrindo o arquivo como um DataFrame

In [50]:
# Exibindo informações sobre o arquivo ODS
FILENAME = '/home/03662232677/workspace-neon/PUCOMEX/SS_PUCOMEX_Jan_2018 a Fev_2019_V2.ods'
show_ods_sheets(FILENAME)

Spreadsheet contains 2 sheet(s).
----------------------------------------
   Sheet name : 'SS'
Size of Sheet : (rows=18172, cols=17)
----------------------------------------
   Sheet name : 'Solution'
Size of Sheet : (rows=158, cols=11)


In [51]:
# Lendo o arquivo ODS com o Pandas
df = read_sheet_ods_as_panda_dataframe(FILENAME)

## Inspecionando e trabalhando o DataFrame

In [52]:
# Mostrando os primeiros registros
df.head()

,ticketid,ownergroup,owner,description,statusdate,externalsystem,serpronivelaten,descricaodemanda,solution,description2,UG_Atendimento,UG_Serviço,relatedreckey,internalpriority,SS_solucaoparausuario,Script_solucaoparausuario,None
0,2017SS/0000940433,UNACDFPUCOMEX,72503360106,"CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",13/03/2018 12:16,EMAIL,3NIVEL,"Bom Dia Tenho dois processos o qual ja voo , n...",,REGRA DE NEGÓCIO,SUNAC,SUNMF,,9,Prezado Usuário\n\nAs cargas estão com a Améri...,#N/DISP,None
1,2017SS/0000973776,UNACDFPUCOMEX,72503360106,"CENTRAL DE SERVIÇOS RFB,CCT - CONTROLE DE CARG...",18/01/2018 23:18,INTERNET,3NIVEL,"Bom dia, Enviei uma nota para o Siscomex e seu...",2016SOL/00396862,ANÁLISE URC,SUNAC,SUNMF,,9,"\n\nPrezado(a) usuário(a),\nProblema resolvido...","Sr.(a) usuário(a), conforme contato telefônic...",None
2,2017SS/0000986377,CAGSBRPUCOMEX,20656866772,"CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",04/01/2018 08:13,EMAIL,2NIVEL,,2016SOL/00399811,SITUAÇÃO NÃO PREVISTA,SUPCD,SUNMF,,9,"Prezado usuario,segue a informação do nosso de...","Sr.(a) usuário(a),conforme informado através d...",None
3,2017SS/0001008950,UNACDFPUCOMEX,72503360106,"CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",10/01/2018 09:28,EMAIL,3NIVEL,"Srs., Fizemos várias tentativas para obter inf...",2017SOL/00003655,REGRA DE NEGÓCIO,SUNAC,SUNMF,,9,"Prezado Usuário,\n\nVocê pode estar tirando as...","\nSr(a). usuário(a),Sua solicitação foi atend...",None
4,2017SS/0001017121,UNACDFPUCOMEX,72503360106,"CENTRAL DE SERVIÇOS RFB,CCT - CONTROLE DE CARG...",10/01/2018 09:31,INTERNET,3NIVEL,Nro da Solicitação: 2017SS/0001012491: Como so...,2017SOL/0000002832,ANÁLISE URC,SUNAC,SUNMF,,9,"Caro usuário,\n\nA DUE 17BR0000079935 já se en...","Sr(a). Usuário(a),É necessário, para análise d...",None


In [53]:
# Selecionando apenas as colunas desejadas
df = df[['description','ownergroup','descricaodemanda','solution','SS_solucaoparausuario','Script_solucaoparausuario']]

In [54]:
# Mudando o nome das colunas
df.columns = ['classificacao','grupo','descricao_demanda','solucao','solucao_usuario','solucao_script']
df.head()

,classificacao,grupo,descricao_demanda,solucao,solucao_usuario,solucao_script
0,"CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",UNACDFPUCOMEX,"Bom Dia Tenho dois processos o qual ja voo , n...",,Prezado Usuário\n\nAs cargas estão com a Améri...,#N/DISP
1,"CENTRAL DE SERVIÇOS RFB,CCT - CONTROLE DE CARG...",UNACDFPUCOMEX,"Bom dia, Enviei uma nota para o Siscomex e seu...",2016SOL/00396862,"\n\nPrezado(a) usuário(a),\nProblema resolvido...","Sr.(a) usuário(a), conforme contato telefônic..."
2,"CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",CAGSBRPUCOMEX,,2016SOL/00399811,"Prezado usuario,segue a informação do nosso de...","Sr.(a) usuário(a),conforme informado através d..."
3,"CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",UNACDFPUCOMEX,"Srs., Fizemos várias tentativas para obter inf...",2017SOL/00003655,"Prezado Usuário,\n\nVocê pode estar tirando as...","\nSr(a). usuário(a),Sua solicitação foi atend..."
4,"CENTRAL DE SERVIÇOS RFB,CCT - CONTROLE DE CARG...",UNACDFPUCOMEX,Nro da Solicitação: 2017SS/0001012491: Como so...,2017SOL/0000002832,"Caro usuário,\n\nA DUE 17BR0000079935 já se en...","Sr(a). Usuário(a),É necessário, para análise d..."


In [55]:
# Exibindo estatísticas do pandas
df.describe()

,classificacao,grupo,descricao_demanda,solucao,solucao_usuario,solucao_script
count,18161,18161,18161,18161,18161,18161
unique,66,34,17059,174,9208,149
top,"CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",ATGSMG,,,,#N/DISP
freq,4216,4654,474,6564,2118,6737


In [56]:
# Explorando os dados da coluna grupo
grupos = df['grupo'].unique()
grupos

array(['UNACDFPUCOMEX', 'CAGSBRPUCOMEX', 'GSDERJCCTR', 'ATGSBRTERC',
       'ATGSMG', 'CAGSBRNOVOEX', 'CAGSBRCOMEX', 'GSDESCPLATAFORMA',
       'GSGSSPCENTRODECOMANDO', 'GSCDSP', 'GSDERJDUEX', 'GSDESCGRIS',
       'GSDESCTALP', 'GSGSDFCENTRODECOMANDO', 'CAGSBRTRIBUT',
       'CAGSRSMDIC', 'GSDERJGATEWAY', 'ATGSSPRFB', 'ATGSPE', 'GSDERSOEA',
       'CACDBRPUCOMEX', 'CAGSBRDWRFB', 'GSDERJCOAD', 'GSDERJSUCE',
       'CAGSBRDRAWBACK', 'CACDBRCOMEX', 'GSCDSPASLGESTAOAT', 'ATGSSP',
       'ATCDMGRAIS', 'GSOPSPSEGI', 'GSOPSPARLDI', 'CACDBRTRIBUT',
       'ATGSBREMAIL', '', None], dtype=object)

In [57]:
# removendo colunas indesejadas de grupo
df = df[df.grupo != '']
df = df.dropna(axis=0, subset=['grupo'])

In [58]:
grupos = df['grupo'].unique()
grupos

array(['UNACDFPUCOMEX', 'CAGSBRPUCOMEX', 'GSDERJCCTR', 'ATGSBRTERC',
       'ATGSMG', 'CAGSBRNOVOEX', 'CAGSBRCOMEX', 'GSDESCPLATAFORMA',
       'GSGSSPCENTRODECOMANDO', 'GSCDSP', 'GSDERJDUEX', 'GSDESCGRIS',
       'GSDESCTALP', 'GSGSDFCENTRODECOMANDO', 'CAGSBRTRIBUT',
       'CAGSRSMDIC', 'GSDERJGATEWAY', 'ATGSSPRFB', 'ATGSPE', 'GSDERSOEA',
       'CACDBRPUCOMEX', 'CAGSBRDWRFB', 'GSDERJCOAD', 'GSDERJSUCE',
       'CAGSBRDRAWBACK', 'CACDBRCOMEX', 'GSCDSPASLGESTAOAT', 'ATGSSP',
       'ATCDMGRAIS', 'GSOPSPSEGI', 'GSOPSPARLDI', 'CACDBRTRIBUT',
       'ATGSBREMAIL'], dtype=object)

In [59]:
# Mostrando os primeiros registros
df.head()

,classificacao,grupo,descricao_demanda,solucao,solucao_usuario,solucao_script
0,"CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",UNACDFPUCOMEX,"Bom Dia Tenho dois processos o qual ja voo , n...",,Prezado Usuário\n\nAs cargas estão com a Améri...,#N/DISP
1,"CENTRAL DE SERVIÇOS RFB,CCT - CONTROLE DE CARG...",UNACDFPUCOMEX,"Bom dia, Enviei uma nota para o Siscomex e seu...",2016SOL/00396862,"\n\nPrezado(a) usuário(a),\nProblema resolvido...","Sr.(a) usuário(a), conforme contato telefônic..."
2,"CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",CAGSBRPUCOMEX,,2016SOL/00399811,"Prezado usuario,segue a informação do nosso de...","Sr.(a) usuário(a),conforme informado através d..."
3,"CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNIC...",UNACDFPUCOMEX,"Srs., Fizemos várias tentativas para obter inf...",2017SOL/00003655,"Prezado Usuário,\n\nVocê pode estar tirando as...","\nSr(a). usuário(a),Sua solicitação foi atend..."
4,"CENTRAL DE SERVIÇOS RFB,CCT - CONTROLE DE CARG...",UNACDFPUCOMEX,Nro da Solicitação: 2017SS/0001012491: Como so...,2017SOL/0000002832,"Caro usuário,\n\nA DUE 17BR0000079935 já se en...","Sr(a). Usuário(a),É necessário, para análise d..."


In [60]:
# Explorando os dados da coluna description
grupos = df['classificacao'].unique()
grupos

array(['CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,REGRA DE \nNEGÓCIO',
       'CENTRAL DE SERVIÇOS RFB,CCT - CONTROLE DE CARGA E TRÂNSITO,ANÁLISE URC',
       'CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,SITUAÇÃO NÃO \nPREVISTA',
       'CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,ANÁLISE URC',
       'CENTRAL DE SERVIÇOS RFB,PORTAL ÚNICO DE COMÉRCIO EXTERIOR,ANÁLISE URC',
       'CENTRAL DE SERVIÇOS RFB,CCT - CONTROLE DE CARGA E TRÂNSITO,APLICAÇÃO',
       'CENTRAL DE SERVIÇOS RFB,PORTAL ÚNICO DE COMÉRCIO EXTERIOR,SITUAÇÃO NÃO \nPREVISTA',
       'CENTRAL DE SERVIÇOS RFB,PORTAL ÚNICO DE COMÉRCIO EXTERIOR,DÚVIDA OU \nINFORMAÇÕES',
       'CENTRAL DE SERVIÇOS RFB,PORTAL ÚNICO DE COMÉRCIO \nEXTERIOR,APLICAÇÃO,APLICAÇÃO VIA FORM-WEB',
       'CENTRAL DE SERVIÇOS RFB,PORTAL ÚNICO DE COMÉRCIO EXTERIOR,AMBIENTE,AMBIENTE \nVIA FORM-WEB',
       'CENTRAL DE SERVIÇOS RFB,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,DÚVIDA OU \nINFORMAÇÕES',
     

In [61]:
# Extraindo o segundo termo após a vírgula
import numpy as np
df['classificacao'] =  (np.where(df['classificacao'].str.contains(','),
                  df['classificacao'].str.split(',').str[1],
                  df['classificacao']))
df.head()

,classificacao,grupo,descricao_demanda,solucao,solucao_usuario,solucao_script
0,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,UNACDFPUCOMEX,"Bom Dia Tenho dois processos o qual ja voo , n...",,Prezado Usuário\n\nAs cargas estão com a Améri...,#N/DISP
1,CCT - CONTROLE DE CARGA E TRÂNSITO,UNACDFPUCOMEX,"Bom dia, Enviei uma nota para o Siscomex e seu...",2016SOL/00396862,"\n\nPrezado(a) usuário(a),\nProblema resolvido...","Sr.(a) usuário(a), conforme contato telefônic..."
2,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,CAGSBRPUCOMEX,,2016SOL/00399811,"Prezado usuario,segue a informação do nosso de...","Sr.(a) usuário(a),conforme informado através d..."
3,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,UNACDFPUCOMEX,"Srs., Fizemos várias tentativas para obter inf...",2017SOL/00003655,"Prezado Usuário,\n\nVocê pode estar tirando as...","\nSr(a). usuário(a),Sua solicitação foi atend..."
4,CCT - CONTROLE DE CARGA E TRÂNSITO,UNACDFPUCOMEX,Nro da Solicitação: 2017SS/0001012491: Como so...,2017SOL/0000002832,"Caro usuário,\n\nA DUE 17BR0000079935 já se en...","Sr(a). Usuário(a),É necessário, para análise d..."


In [62]:
grupos = df['classificacao'].unique()
print(len(grupos))
grupos

24


array(['DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO',
       'CCT - CONTROLE DE CARGA E TRÂNSITO',
       'PORTAL ÚNICO DE COMÉRCIO EXTERIOR',
       'PORTAL ÚNICO DE COMÉRCIO \nEXTERIOR', 'DÚVIDA',
       'SITUAÇÃO NÃO PREVISTA', 'APLICAÇÃO', 'DA - DESPACHO ADUANEIRO',
       'TA-LPCO - TRATAMENTO ADMIN', 'ANÁLISE URC',
       'CA - CONFERÊNCIA ADUANEIRA EXPORTAÇÃO', 'AMBIENTE',
       'DÚVIDA OU INFORMAÇÕES', 'GR – GERENCIAMENTO DE RISCO ALEATÓRIOS',
       'DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO - MDIC',
       'PROSPECÇÃO - CLASSIF - CLASSIFICAÇÃO FISCAL DE \nMERCADORIAS RFB',
       'PORTAL ÚNICO SISCOMEX RFB \nINTRANET(PUCOMEX)',
       'PORTAL ÚNICO SISCOMEX RFB INTRANET(PUCOMEX)',
       'PORTAL ÚNICO SISCOMEX ANUENTES INTERNET (PUCOMEX)',
       'CADASTRO DE INTERVENIENTES - RFB',
       'OPERADOR ECONÔMICO AUTORIZADO (OEA) - RFB',
       'DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO - \nRFB',
       'DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO - RFB',
       'VICOMEX - VISÃO INTEGRADA DO COMÉRCI

In [63]:
df['classificacao'] = df['classificacao'].replace(r'\n','', regex=True)
grupos = df['classificacao'].unique()
print(len(grupos))
grupos

21


array(['DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO',
       'CCT - CONTROLE DE CARGA E TRÂNSITO',
       'PORTAL ÚNICO DE COMÉRCIO EXTERIOR', 'DÚVIDA',
       'SITUAÇÃO NÃO PREVISTA', 'APLICAÇÃO', 'DA - DESPACHO ADUANEIRO',
       'TA-LPCO - TRATAMENTO ADMIN', 'ANÁLISE URC',
       'CA - CONFERÊNCIA ADUANEIRA EXPORTAÇÃO', 'AMBIENTE',
       'DÚVIDA OU INFORMAÇÕES', 'GR – GERENCIAMENTO DE RISCO ALEATÓRIOS',
       'DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO - MDIC',
       'PROSPECÇÃO - CLASSIF - CLASSIFICAÇÃO FISCAL DE MERCADORIAS RFB',
       'PORTAL ÚNICO SISCOMEX RFB INTRANET(PUCOMEX)',
       'PORTAL ÚNICO SISCOMEX ANUENTES INTERNET (PUCOMEX)',
       'CADASTRO DE INTERVENIENTES - RFB',
       'OPERADOR ECONÔMICO AUTORIZADO (OEA) - RFB',
       'DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO - RFB',
       'VICOMEX - VISÃO INTEGRADA DO COMÉRCIO EXTERIOR - MDIC'],
      dtype=object)

## Preparação dos dados

In [64]:
# Estrutura para armazenar pontuações
tbl = dict.fromkeys(i for i in range(sys.maxunicode) if unicodedata.category(chr(i)).startswith('P'))
# Função para remover pontuações das sentenças
def remove_punctuation(text):
    return text.translate(tbl)

In [65]:
# Lista de palavras para inluir com regex
INCLUDE_WORDS_BY_REGEX = [r"^[A-Za-zçãàáâéêíóôõúüÂÃÁÀÉÊÍÓÔÕÚÜÇ'º°ª\.]*$",r"^[0-9\-/\\\.]*$"]
def include_words_by_regex(words):
    # Remove regex words
    words_aux = []
    for word in words:
        add = False
        for regex_exp in INCLUDE_WORDS_BY_REGEX:
            if re.compile(regex_exp).match(word):
                add = True
        if add:
            words_aux.append(word)
    return words_aux       
# Padroes regex para serem removidos do texto original
REMOVE_WORDS_BY_REGEX = [r"\S*@\S*\s?", # Email
    r"(http:\/\/www\.|https:\/\/www\.|http:\/\/|https:\/\/)?[a-z0-9]+([\-\.]{1}[a-z0-9]+)*\.[a-z]{2,5}(:[0-9]{1,5})?(\/[a-z0-9]*)*(\/)?", # Site
    r"([a-z0-9]+(\/)[a-z0-9]+)", # Protocolo
    r"(\(?\d{2,4}\)?)??[\-\s\.]?(\d{3,5})[\.\-]\d{4,5}", # Telefone
    r"\d{2}[\-\s\.]\d{4}", # Ramal
    r"([0-9]{2}[\.]?[0-9]{3}[\.]?[0-9]{3}[\/]?[0-9]{4}[-]?[0-9]{2})|([0-9]{3}[\.]?[0-9]{3}[\.]?[0-9]{3}[-]?[0-9]{2})", # CPF e CNPJ
    r"\d+" # Numeros
    #r"e-?mail:?" # texto email/e-mail
]
def remove_words_by_regex(text):
    for regex_exp in REMOVE_WORDS_BY_REGEX:
        text = re.sub(re.compile(regex_exp),"",text)
    return text
# Padroes regex para remoção do conteúdo a seguir
REMOVE_TEXT_AFTER_REGEX = [
        r"o acesso ao conte(u|ú)do desta mensagem",
        r"aviso legal",
        r"esta mensagem (e|é) destinada exclusivamente",
        r"legal advice",
        r"this message is exclusively destined"
]
def remove_text_after_regex(text):
    for regex_exp in REMOVE_TEXT_AFTER_REGEX:
        m = re.search(regex_exp, text)
        if m:
            text = text[:m.start()]
    return text

In [66]:
text = """"
srs., fizemos várias tentativas para obter informações em relação à adesão 
ao portal due, alguns exemplos: primeiramente acessamos o site: 
http://www.serpro.gov.br/menu/contato/cliente/ depois por telefone: 
0800-728-2323 e também nos contatos: 
sccd@serpro.gov.br<mailto:sccd@serpro.gov.br> 
http://comexresponde.comexbrasil.gov.br/ - protocolo: 30696 em uma das 
orientações recebidas (protocolo nº 2017ss/0000973710), nos informaram que 
o acesso seria através do mdic / comex, procedemos de acordo, porém não 
houve retorno. por favor, precisamos do contato urgente para alinhamento. 
favor contatar: fernanda cezar no telefone:3646-2256 / e-mail: 
fernanda.cezar@bradesco.com.br gratos. atenciosamente. banco bradesco s/a 
4861 / droc – departamento de relacionamento operacional com clientes setor 
apoio a projetos patrícia vidal tel: (11) 2646-3320 r: 31-3320 
classificação: interna “o acesso ao conteúdo desta mensagem está 
autorizado, exclusivamente, a colaboradores da organização bradesco. a 
reprodução a pessoas não enquadradas deve ser autorizada pelo gestor da 
informação, para identificá-lo contate o remetente". 
[cid:image164146.png@3690fe11.49984c89] aviso legal
...esta mensagem é destinada exclusivamente para a(s) pessoa(s) a quem é 
dirigida, podendo conter informação confidencial e/ou legalmente 
privilegiada. se você não for destinatário desta mensagem, desde já fica 
notificado de abster-se a divulgar, copiar, distribuir, examinar ou, de 
qualquer forma, utilizar a informação contida nesta mensagem, por ser 
ilegal. caso você tenha recebido esta mensagem por engano, pedimos que nos 
retorne este e-mail, promovendo, desde logo, a eliminação do seu conteúdo 
em sua base de dados, registros ou sistema de controle. fica desprovida de 
eficácia e validade a mensagem que contiver vínculos obrigacionais, 
expedida por quem não detenha poderes de representação. legal advice
...this message is exclusively destined for the people to whom it is 
directed, and it can bear private and/or legally exceptional information. 
if you are not addressee of this message, since now you are advised to not 
release, copy, distribute, check or, otherwise, use the information 
contained in this message, because it is illegal. if you received this 
message by mistake, we ask you to return this email, making possible, as 
soon as possible, the elimination of its contents of your database, 
registrations or controls system. the message that bears any mandatory 
links, issued by someone who has no representation powers, shall be null or 
void. [screenshot 1 saved as attachment image1641461512409665054.png by 
e-mail listener.]
"""
text = remove_words_by_regex(text.lower())
print(text)
text2 = remove_text_after_regex(text)
print(text2)

"
srs., fizemos várias tentativas para obter informações em relação à adesão 
ao portal due, alguns exemplos: primeiramente acessamos o site: 
 depois por telefone: 
 e também nos contatos: 

 - protocolo:  em uma das 
orientações recebidas (protocolo nº ), nos informaram que 
o acesso seria através do mdic / comex, procedemos de acordo, porém não 
houve retorno. por favor, precisamos do contato urgente para alinhamento. 
favor contatar: fernanda cezar no telefone: / e-mail: 
gratos. atenciosamente. banco bradesco  
 / droc – departamento de relacionamento operacional com clientes setor 
apoio a projetos patrícia vidal tel:  r:  
classificação: interna “o acesso ao conteúdo desta mensagem está 
autorizado, exclusivamente, a colaboradores da organização bradesco. a 
reprodução a pessoas não enquadradas deve ser autorizada pelo gestor da 
informação, para identificá-lo contate o remetente". 
aviso legal
...esta mensagem é destinada exclusivamente para a(s) pessoa(s) a quem é 
dirigida, p

In [67]:
spacy_pln = spacy.load('pt_core_news_sm')
words = [token.lemma_ for token in spacy_pln(text2)]
print (words)

['"', '\n', 'sr', '.', ',', 'fazer', 'vários', 'tentativo', 'parir', 'obter', 'informação', 'em', 'relação', 'à', 'adesão', '\n', 'o', 'o', 'portal', 'due', ',', 'algum', 'exemplo', ':', 'primeiramente', 'acessamos', 'o', 'site', ':', '\n ', 'depois', 'por', 'telefonar', ':', '\n ', 'e', 'também', 'o', 'contatos', ':', '\n\n ', '-', 'protocolo', ':', ' ', 'em', 'umar', 'das', '\n', 'orientação', 'recebido', '(', 'protocolo', 'nº', ')', ',', 'o', 'informar', 'que', '\n', 'o', 'acesso', 'seriar', 'através', 'do', 'mdic', '/', 'comex', ',', 'proceder', 'de', 'acordar', ',', 'porém', 'não', '\n', 'haver', 'retornar', '.', 'por', 'favor', ',', 'precisar', 'do', 'contato', 'urgente', 'parir', 'alinhamento', '.', '\n', 'favor', 'contatar', ':', 'fernanda', 'cezar', 'o', 'telefonar', ':', '/', 'e', '-', 'mail', ':', '\n', 'grato', '.', 'atenciosamente', '.', 'banco', 'bradesco', ' \n ', '/', 'droc', '–', 'departamento', 'de', 'relacionamento', 'operacional', 'com', 'cliente', 'setor', '\n', 'a

In [68]:
spacy_pln = spacy.load('pt_core_news_sm')
# Função para realizar o pré-processamento do texto livre
# Inicializa o stemmer
#stemmer = LancasterStemmer() 
stemmer = RSLPStemmer()
# Stop words para o Português
portuguese_stops = set(stopwords.words('portuguese'))
portuguese_stops.update('é')
REMOVER = ['attach','email']
LIMITE_TAMANHO_PALAVRA = 2
# Dicionátio que armazena as pontuações a serem removidas
def preprocessing(content):
    # Converte para minúsculo
    content = content.lower()
    # Remove elementos do texto via REGEX
    content = remove_words_by_regex(content)
    content = remove_text_after_regex(content)
    #Remover pontuação
    content = remove_punctuation(content)
    # Tokenização
    #words = tokenize.word_tokenize(content, language='portuguese')
    words = [token.lemma_ for token in spacy_pln(content)]
    # Remover acentuação
    words = [normalize('NFKD', word).encode('ASCII', 'ignore').decode('ASCII') for word in words]
    # Remove stopwords
    words = [word for word in words if word.lower() not in portuguese_stops] 
    # Stemming de cada palavra
    #words = [stemmer.stem(w) for w in words if w.strip()] 
    # Remove palavras menores que um limite
    words = [word for word in words if len(word) > LIMITE_TAMANHO_PALAVRA] 
    # Remove by regex
    words = include_words_by_regex(words)
    # Remove specific words
    words = [word for word in words if word not in REMOVER] 
    return ' '.join(words)

In [69]:
# Imprimindo alguns exemplos
for i in range(1,5):
    print('============================================================================')
    sample = df['descricao_demanda'][i]
    print(sample)
    print('----------------------------------------------------------------------------')
    print(preprocessing(sample))

Bom dia, Enviei uma nota para o Siscomex e seu peso aferido ficou com um 
valor diferente. O peso aferido enviado foi 49920.000 e o que aparece ao 
consultar no Siscomex é 67140000.000. Segue em anexo o xml enviado e um 
print da tela do Siscomex.
----------------------------------------------------------------------------
bom dia enviar umar noto parir siscomex pesar aferir ficar valor diferente pesar aferir enviar ser aparecer consultar siscomex ser seguir anexar xml enviar print tela siscomex

----------------------------------------------------------------------------

Srs., Fizemos várias tentativas para obter informações em relação à adesão 
ao Portal DUE, alguns exemplos: Primeiramente acessamos o site: 
http://www.serpro.gov.br/menu/contato/cliente/ Depois por telefone: 
0800-728-2323 E também nos contatos: 
sccd@serpro.gov.br<mailto:sccd@serpro.gov.br> 
http://comexresponde.comexbrasil.gov.br/ - protocolo: 30696 Em uma das 
orientações recebidas (protocolo nº 2017SS/0000973710

In [70]:
# Aplicando o pre-processamento
print('Iniciando a preparação das colunas textuais...')
start_time = time.time()
df['descricao_demanda2'] = df['descricao_demanda'].apply(preprocessing)
print('Processou a coluna descricao_demanda2 em %.2f segundos' % (time.time() - start_time))
start_time = time.time()
df['solucao_usuario2'] = df['solucao_usuario'].apply(preprocessing)
print('Processou a coluna solucao_usuario2 em %.2f segundos' % (time.time() - start_time))
start_time = time.time()
df['solucao_script2'] = df['solucao_script'].apply(preprocessing)
print('Processou a coluna solucao_script2 em %.2f segundos' % (time.time() - start_time))
df.head()

Iniciando a preparação das colunas textuais...
Processou a coluna descricao_demanda2 em 342.38 segundos
Processou a coluna solucao_usuario2 em 265.41 segundos
Processou a coluna solucao_script2 em 207.55 segundos


,classificacao,grupo,descricao_demanda,solucao,solucao_usuario,solucao_script,descricao_demanda2,solucao_usuario2,solucao_script2
0,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,UNACDFPUCOMEX,"Bom Dia Tenho dois processos o qual ja voo , n...",,Prezado Usuário\n\nAs cargas estão com a Améri...,#N/DISP,bom dia ter dois processo voar nao conseguir c...,prezar usuario carga estar american airlines t...,
1,CCT - CONTROLE DE CARGA E TRÂNSITO,UNACDFPUCOMEX,"Bom dia, Enviei uma nota para o Siscomex e seu...",2016SOL/00396862,"\n\nPrezado(a) usuário(a),\nProblema resolvido...","Sr.(a) usuário(a), conforme contato telefônic...",bom dia enviar umar noto parir siscomex pesar ...,prezadoa usuarioa problema resolver favor test...,usuarioa conformar contato telefonico parir pr...
2,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,CAGSBRPUCOMEX,,2016SOL/00399811,"Prezado usuario,segue a informação do nosso de...","Sr.(a) usuário(a),conforme informado através d...",,prezar usuariosegue informacao desenvolvimento...,usuarioaconforme informar atraves contato tele...
3,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,UNACDFPUCOMEX,"Srs., Fizemos várias tentativas para obter inf...",2017SOL/00003655,"Prezado Usuário,\n\nVocê pode estar tirando as...","\nSr(a). usuário(a),Sua solicitação foi atend...",fazer varios tentativo parir obter informacao ...,prezar usuario voce poder estar tirar suar duv...,usuarioasua solicitacao ser iosamentecentral s...
4,CCT - CONTROLE DE CARGA E TRÂNSITO,UNACDFPUCOMEX,Nro da Solicitação: 2017SS/0001012491: Como so...,2017SOL/0000002832,"Caro usuário,\n\nA DUE 17BR0000079935 já se en...","Sr(a). Usuário(a),É necessário, para análise d...",nro solicitacao comer solicitar nao conseguir ...,caro usuario due encontrar averbar,usuarioae necessario parir analise situacao ac...


In [71]:
# Combinando a classificação e grupo
df['classificacao_grupo'] = df['classificacao'].astype(str) + ' - ' + df['grupo'].astype(str)

In [72]:
# Reorganizando a ordem das colunas
df = df[['classificacao_grupo','classificacao', 'grupo', 'descricao_demanda2','solucao','solucao_usuario2','solucao_script2','descricao_demanda','solucao_usuario','solucao_script']]
df.head()

,classificacao_grupo,classificacao,grupo,descricao_demanda2,solucao,solucao_usuario2,solucao_script2,descricao_demanda,solucao_usuario,solucao_script
0,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO - UNACDF...,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,UNACDFPUCOMEX,bom dia ter dois processo voar nao conseguir c...,,prezar usuario carga estar american airlines t...,,"Bom Dia Tenho dois processos o qual ja voo , n...",Prezado Usuário\n\nAs cargas estão com a Améri...,#N/DISP
1,CCT - CONTROLE DE CARGA E TRÂNSITO - UNACDFPUC...,CCT - CONTROLE DE CARGA E TRÂNSITO,UNACDFPUCOMEX,bom dia enviar umar noto parir siscomex pesar ...,2016SOL/00396862,prezadoa usuarioa problema resolver favor test...,usuarioa conformar contato telefonico parir pr...,"Bom dia, Enviei uma nota para o Siscomex e seu...","\n\nPrezado(a) usuário(a),\nProblema resolvido...","Sr.(a) usuário(a), conforme contato telefônic..."
2,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO - CAGSBR...,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,CAGSBRPUCOMEX,,2016SOL/00399811,prezar usuariosegue informacao desenvolvimento...,usuarioaconforme informar atraves contato tele...,,"Prezado usuario,segue a informação do nosso de...","Sr.(a) usuário(a),conforme informado através d..."
3,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO - UNACDF...,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,UNACDFPUCOMEX,fazer varios tentativo parir obter informacao ...,2017SOL/00003655,prezar usuario voce poder estar tirar suar duv...,usuarioasua solicitacao ser iosamentecentral s...,"Srs., Fizemos várias tentativas para obter inf...","Prezado Usuário,\n\nVocê pode estar tirando as...","\nSr(a). usuário(a),Sua solicitação foi atend..."
4,CCT - CONTROLE DE CARGA E TRÂNSITO - UNACDFPUC...,CCT - CONTROLE DE CARGA E TRÂNSITO,UNACDFPUCOMEX,nro solicitacao comer solicitar nao conseguir ...,2017SOL/0000002832,caro usuario due encontrar averbar,usuarioae necessario parir analise situacao ac...,Nro da Solicitação: 2017SS/0001012491: Como so...,"Caro usuário,\n\nA DUE 17BR0000079935 já se en...","Sr(a). Usuário(a),É necessário, para análise d..."


In [73]:
FILENAME = 'dataframe.pickle'
df.to_pickle(FILENAME)
print('arquivo salvo com sucesso!')

arquivo salvo com sucesso!


FIM